# Treinamento e Otimização de Modelos para o Titanic

## 1. Configuração Inicial e Imports

Importação das bibliotecas necessárias e configuração do ambiente, como logs e avisos.

In [ ]:
import pandas as pd
import joblib
import logging
import os
import numpy as np
import json
import time
import warnings
import sys
import optuna
from pathlib import Path
from typing import Dict, Any, List, Tuple, Type, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
optuna.logging.set_verbosity(optuna.logging.WARNING)

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", handlers=[logging.StreamHandler(sys.stdout)])
logger = logging.getLogger(__name__)

## 2. Configurações do Projeto

Configuração de constantes e caminhos de diretórios necessários para o projeto.

In [ ]:
Model = Type[Any]

BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
PROCESSED_DATA_DIR = DATA_DIR / "processed"
ARTIFACTS_DIR = BASE_DIR / "artifacts"
MODELS_DIR = ARTIFACTS_DIR / "models"
REPORTS_DIR = BASE_DIR / "reports"
FIGURES_DIR = REPORTS_DIR / "figures" 

PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_FILE = RAW_DATA_DIR / "train.csv"
TEST_FILE = RAW_DATA_DIR / "test.csv"
SUBMISSION_FILE = DATA_DIR / "submission.csv"
BEST_MODEL_FILE = ARTIFACTS_DIR / "ensemble_model.joblib"
FE_PIPELINE_FILE = ARTIFACTS_DIR / "fe_pipeline.joblib"
METRICS_FILE = ARTIFACTS_DIR / "training_metrics.json"

TARGET_COLUMN: str = "Survived"

MODELS_TO_TUNE: Dict[str, Model] = {
    "RandomForest": RandomForestClassifier,
    "GradientBoosting": GradientBoostingClassifier,
    "XGBClassifier": XGBClassifier,
    "LGBMClassifier": LGBMClassifier,
    "SVC": SVC,
}

logger.info(f"Diretório base configurado para: {BASE_DIR}")

## 3. Engenharia de Features (Pré-processamento)

Esta função, antes em `processing/preprocessor.py`, é responsável por criar novas features e tratar valores ausentes a partir do DataFrame bruto.

In [ ]:
def feature_engineering(df: pd.DataFrame, bins: Optional[Dict[str, list]] = None) -> pd.DataFrame:
    df_copy = df.copy()
    
    # tratamento de valores ausentes
    if 'Age' in df_copy.columns:
        df_copy['Age'].fillna(df_copy['Age'].median(), inplace=True)
    if 'Fare' in df_copy.columns:
        df_copy['Fare'].fillna(df_copy['Fare'].median(), inplace=True)
    if 'Embarked' in df_copy.columns:
        df_copy['Embarked'].fillna(df_copy['Embarked'].mode()[0], inplace=True)
    
    # criação da feature 'Title'
    df_copy['Title'] = df_copy['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    common_titles = {"Mr", "Miss", "Mrs", "Master"}
    df_copy['Title'] = df_copy['Title'].apply(lambda x: x if x in common_titles else 'Other')

    # criação das features 'FamilySize' e 'IsAlone'
    df_copy['FamilySize'] = df_copy['SibSp'] + df_copy['Parch'] + 1
    df_copy['IsAlone'] = (df_copy['FamilySize'] == 1).astype(int)
    
    # discretização de 'Age' e 'Fare'
    if bins:
        df_copy['AgeGroup'] = pd.cut(df_copy['Age'], bins=bins['AgeBins'], labels=False, include_lowest=True)
        df_copy['FareBin'] = pd.cut(df_copy['Fare'], bins=bins['FareBins'], labels=False, include_lowest=True)
    else:
        df_copy['AgeGroup'] = pd.cut(df_copy['Age'], bins=[0, 12, 18, 60, np.inf], labels=False, include_lowest=True)
        try:
            df_copy['FareBin'] = pd.qcut(df_copy['Fare'], 4, labels=False, duplicates='drop')
        except ValueError:
            # fallback para o caso de não ser possível criar quantis
            df_copy['FareBin'] = pd.cut(df_copy['Fare'], 4, labels=False, include_lowest=True)

    return df_copy

## 4. Definição dos Pipelines de Modelagem

A função `create_modeling_pipeline` define a estrutura de pré-processamento (scaling para features numéricas e one-hot encoding para categóricas) e o anexa a um classificador para formar um pipeline completo.

In [ ]:
TrainResult = Tuple[str, Pipeline, float, float, Dict[str, Any]]

def create_modeling_pipeline(model: Any) -> Pipeline:
    numeric_features = ["Age", "Fare", "FamilySize"]
    categorical_features = [
        "Embarked",
        "Sex",
        "Pclass",
        "Title",
        "IsAlone",
        "AgeGroup",
        "FareBin",
    ]

    numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])
    categorical_transformer = Pipeline(
        steps=[("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ],
        remainder="drop",
    )
    return Pipeline(steps=[("preprocessor", preprocessor), ("classifier", model)])

## 5. Otimização de Hiperparâmetros com Optuna

A função `get_objective_function` cria uma função objetivo para o Optuna. Esta função define o espaço de busca dos hiperparâmetros para cada modelo e retorna a acurácia média da validação cruzada, que o Optuna tentará maximizar.

In [ ]:
def get_objective_function(model_name: str, X: pd.DataFrame, y: pd.Series, models_to_tune: Dict[str, Any]):
    def objective(trial: optuna.Trial) -> float:
        if model_name == "RandomForest":
            params = {
                "n_estimators": trial.suggest_int("n_estimators", 100, 500),
                "max_depth": trial.suggest_int("max_depth", 5, 20),
                "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            }
            model_instance = models_to_tune[model_name](random_state=42, n_jobs=1, **params)
        elif model_name == "GradientBoosting":
            params = {
                "n_estimators": trial.suggest_int("n_estimators", 100, 500),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
            }
            model_instance = models_to_tune[model_name](random_state=42, **params)
        elif model_name == "SVC":
            params = {"C": trial.suggest_float("C", 0.1, 10.0)}
            model_instance = models_to_tune[model_name](random_state=42, probability=True, **params)
        elif model_name == "XGBClassifier":
            params = {
                "n_estimators": trial.suggest_int("n_estimators", 100, 500),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
            }
            model_instance = models_to_tune[model_name](random_state=42, eval_metric="logloss", use_label_encoder=False, **params)
        elif model_name == "LGBMClassifier":
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 500),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                'num_leaves': trial.suggest_int('num_leaves', 20, 50),
            }
            model_instance = models_to_tune[model_name](random_state=42, verbose=-1, **params)
        else:
            raise ValueError(f"Modelo {model_name} não suportado.")

        pipeline = create_modeling_pipeline(model_instance)
        score = cross_val_score(
            pipeline, X, y, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring="accuracy"
        ).mean()
        return score
    return objective

## 6. Função de Treinamento de Modelo Individual

`train_single_model` encapsula o processo de otimização para um único modelo: cria um estudo do Optuna, executa a otimização, treina o modelo final com os melhores parâmetros e retorna seus resultados.

In [ ]:
def train_single_model(model_name: str, X: pd.DataFrame, y: pd.Series, n_trials: int, models_to_tune: Dict[str, Any]) -> TrainResult:
    study = optuna.create_study(direction="maximize")
    objective_fn = get_objective_function(model_name, X, y, models_to_tune)
    study.optimize(objective_fn, n_trials=n_trials, n_jobs=1, show_progress_bar=False)

    best_params = study.best_params
    logger.info(f"Otimização para {model_name} concluída. Melhores parâmetros: {best_params}")

    model_class = models_to_tune[model_name]
    if model_name == "SVC":
        best_params["probability"] = True
    elif model_name == "LGBMClassifier":
        best_params["verbose"] = -1

    final_model_instance = model_class(**best_params, random_state=42)
    final_pipeline = create_modeling_pipeline(final_model_instance)

    cv_scores = cross_val_score(
        final_pipeline, X, y, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring="accuracy"
    )
    mean_accuracy, std_dev = np.mean(cv_scores), np.std(cv_scores)
    
    logger.info(f"Modelo {model_name} validado. Acurácia: {mean_accuracy:.4f} (+/- {std_dev:.4f})")
    final_pipeline.fit(X, y)
    
    return model_name, final_pipeline, mean_accuracy, std_dev, best_params

## 7. Execução Principal do Treinamento

Esta é a célula principal que orquestra todo o processo. Primeiro, definimos os parâmetros de treinamento.

In [ ]:
MODELS_TO_TRAIN_LIST = list(MODELS_TO_TUNE.keys())
N_TRIALS = 1 # número de tentativas de otimização

### 7.1. Carga e Processamento dos Dados

Carregamos os dados de treino e aplicamos o pipeline de engenharia de features.

In [ ]:
start_time = time.perf_counter()
training_metrics: Dict[str, Any] = {"individual_models": {}}

df = pd.read_csv(str(TRAIN_FILE))
logger.info("Aplicando engenharia de features...")

fe_pipeline = Pipeline(steps=[('feature_engineering', FunctionTransformer(feature_engineering))])

X = df.drop(TARGET_COLUMN, axis=1)
y = df[TARGET_COLUMN]
X_processed = fe_pipeline.fit_transform(X)

joblib.dump(fe_pipeline, FE_PIPELINE_FILE)
logger.info(f"Pipeline de engenharia de features salvo em: {FE_PIPELINE_FILE}")

### 7.2. Treinamento Paralelo dos Modelos Base

Utilizamos `ThreadPoolExecutor` para treinar e otimizar os modelos selecionados em paralelo, acelerando significativamente o processo.

In [ ]:
models_to_tune_filtered = {k: v for k, v in MODELS_TO_TUNE.items() if k in MODELS_TO_TRAIN_LIST}
logger.info(f"Modelos a serem treinados: {list(models_to_tune_filtered.keys())}")
logger.info(f"Número de trials por modelo: {N_TRIALS}")

results: List[TrainResult] = []
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = {
        executor.submit(train_single_model, name, X_processed.copy(), y.copy(), N_TRIALS, models_to_tune_filtered): name
        for name in models_to_tune_filtered
    }

    for future in as_completed(futures):
        results.append(future.result())


### 7.3. Criação e Treinamento do Modelo Ensemble

Selecionamos os 3 melhores modelos base com base na acurácia da validação cruzada. Em seguida, usamos esses modelos como estimadores para um `StackingClassifier`, que é treinado para se tornar o nosso modelo final.

In [ ]:
if not results:
    logger.error("Nenhum modelo foi treinado. Verifique as configurações.")
else:
    # seleciona os 3 melhores modelos
    top_models: List[TrainResult] = sorted(results, key=lambda item: item[2], reverse=True)[:3]

    for name, pipeline, score, std_dev, params in top_models:
        model_path = MODELS_DIR / f"{name.lower()}_model.joblib"
        joblib.dump(pipeline, model_path)
        training_metrics["individual_models"][name] = {"accuracy": score, "std_dev": std_dev, "params": params, "path": str(model_path)}
        logger.info(f"Modelo {name} salvo com acurácia {score:.4f} (+/- {std_dev:.4f}).")
    
    logger.info("Iniciando treinamento do modelo Stacking Ensemble...")
    
    stacking_estimators = [(name.lower(), model) for name, model, _, _, _ in top_models]
    meta_model = LogisticRegression(random_state=42)
    stacking_clf = StackingClassifier(estimators=stacking_estimators, final_estimator=meta_model, cv=5, n_jobs=-1)

    # valida o modelo
    cv_scores = cross_val_score(stacking_clf, X_processed, y, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring="accuracy", n_jobs=-1)
    mean_score, std_score = np.mean(cv_scores), np.std(cv_scores)
    logger.info(f"Acurácia do Stacking: {mean_score:.4f} (+/- {std_score:.4f})")

    training_metrics["ensemble_model"] = {"type": "StackingClassifier", "accuracy": mean_score, "std_dev": std_score, "estimators": [name for name, _, _, _, _ in top_models], "final_estimator": type(meta_model).__name__}

    logger.info("Treinando o Stacking final com todos os dados...")
    stacking_clf.fit(X_processed, y)
    joblib.dump(stacking_clf, BEST_MODEL_FILE)
    logger.info(f"Melhor modelo (Ensemble) salvo em: {BEST_MODEL_FILE}")
    
    # salva as métricas
    training_metrics["total_training_time"] = time.perf_counter() - start_time
    with open(METRICS_FILE, "w", encoding='utf-8') as f:
        json.dump(training_metrics, f, indent=4)
    logger.info(f"Métricas de treinamento salvas em: {METRICS_FILE}")

## 8. Geração do Arquivo de Submissão para o Kaggle

Com o melhor modelo treinado (o ensemble), agora o usamos para fazer predições no conjunto de dados de teste (`test.csv`) e formatamos o resultado no arquivo `submission.csv`, pronto para ser enviado ao Kaggle.

In [ ]:
if BEST_MODEL_FILE.exists() and FE_PIPELINE_FILE.exists():
    logger.info("Iniciando a geração do arquivo de submissão...")
    
    df_test = pd.read_csv(str(TEST_FILE))
    passenger_ids = df_test['PassengerId']
    
    fe_pipeline_loaded = joblib.load(FE_PIPELINE_FILE)
    final_model_loaded = joblib.load(BEST_MODEL_FILE)
    
    X_test_processed = fe_pipeline_loaded.transform(df_test)
    
    predictions = final_model_loaded.predict(X_test_processed)
    
    submission_df = pd.DataFrame({'PassengerId': passenger_ids, 'Survived': predictions})
    submission_df.to_csv(SUBMISSION_FILE, index=False)
    
    logger.info(f"Arquivo de submissão salvo com sucesso em: {SUBMISSION_FILE}")
    display(submission_df.head())
else:
    logger.error("Artefatos de modelo ou pipeline não encontrados. Não foi possível gerar a submissão.")

logger.info("Processo de treinamento e submissão concluído.")